Installing pyspark 

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 65kB/s 
     |████████████████████████████████| 204kB 37.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=38aa20d2f0c29ffc635e9926ed48592580b6267e8b35418546ce83b92aedf470
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


Mounting Google Drive


In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


intializing Spark Session

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('RDD_Example').getOrCreate()

In [4]:
distFile = spark.sparkContext.textFile("/content/gdrive/My Drive/Colab Notebooks/data/Air_Quality_dataset.csv")
lines = distFile.flatMap(lambda line: line.split("\t"))
row_data = lines.map(lambda row: row.split(","))

Find the city which has maximum average NH3 pollutant value


In [5]:
    row_NH3 = row_data.filter(lambda row: row[7] == "NH3")
    real_row_NH3 = row_NH3.filter(lambda row: row[4] != "NA")
    pair = real_row_NH3.map(lambda row: (int(row[4]), row))
    sorted_pairs = pair.sortByKey(False)
    print("The city which has maximum average NH3 pollutant value is " + sorted_pairs.first()[1][2])

The city which has maximum average NH3 pollutant value is Nagpur


Find the number of results for each pollutant type

In [6]:
    pair = row_data.map(lambda row: (row[7], row))
    row_count = pair.countByKey()
 
    for pollutant_type,count in row_count.items():
        print("Number of results for " + pollutant_type + " : " +str(count))

Number of results for PM2.5 : 127
Number of results for PM10 : 108
Number of results for NO2 : 129
Number of results for NH3 : 92
Number of results for SO2 : 121
Number of results for CO : 128
Number of results for OZONE : 119


Find the top 10 resulting rows which have maximum pollutant level in CO and store the result in a text file.


In [7]:
    row_CO = row_data.filter(lambda row: row[7] == "CO")
    real_row_CO = row_CO.filter(lambda row: row[5] != "NA")
    pair = real_row_CO.map(lambda row: (int(row[5]), row))
    sorted_pairs = pair.sortByKey(False)
    top_ten = sorted_pairs.take(10)
    
    top_ten_row = []
 
    for key,value in top_ten:
        top_ten_row.append(value)

    spark.sparkContext.parallelize(top_ten_row).saveAsTextFile("/content/gdrive/My Drive/Colab Notebooks/output/top_ten_max_CO")
